<a href="https://colab.research.google.com/github/windyrahayu45/Proyek-Analisis-Sentimen/blob/main/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import time
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Pastikan stopwords bahasa Indonesia tersedia
nltk.download("stopwords")
stop_words = set(stopwords.words("indonesian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# Shop ID dan Item ID dari URL produk Shopee
SHOP_ID = 468947139
ITEM_ID = 23878205368
MAX_REVIEWS = 10000  # Target minimal 10.000 data
LIMIT = 50

# Set Header & Cookies dari Browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Referer": f"https://shopee.co.id/product/{SHOP_ID}/{ITEM_ID}/",
}

# Fungsi scraping dengan session
def scrape_shopee_reviews(shop_id, item_id, max_reviews=10000):
    session = requests.Session()
    session.headers.update(HEADERS)

    reviews = []
    offset = 0

    while len(reviews) < max_reviews:
        print(f"🔄 Mengambil data dari offset {offset}...")
        url = f"https://shopee.co.id/api/v2/item/get_ratings?itemid={item_id}&shopid={shop_id}&offset={offset}&limit={LIMIT}&type=0"

        try:
            response = session.get(url)
            if response.status_code != 200:
                print("⚠️ Gagal mengambil data, status code:", response.status_code)
                break

            data = response.json()
            if "data" in data and "ratings" in data["data"]:
                for review in data["data"]["ratings"]:
                    reviews.append({
                        "Username": review.get("author_username", ""),
                        "Rating": review.get("rating_star", 0),
                        "Komentar": review.get("comment", "")
                    })

            offset += LIMIT
            if len(data["data"]["ratings"]) < LIMIT:
                break

            time.sleep(1)

        except Exception as e:
            print("⚠️ Terjadi kesalahan:", str(e))
            break

    return pd.DataFrame(reviews)

# Jalankan scraping
df_reviews = scrape_shopee_reviews(SHOP_ID, ITEM_ID, MAX_REVIEWS)

# Simpan hasil scraping sebelum preprocessing
df_reviews.to_csv("shopee_reviews_raw.csv", index=False)
print("✅ Data mentah telah disimpan sebagai 'shopee_reviews_raw.csv'.")

🔄 Mengambil data dari offset 0...
🔄 Mengambil data dari offset 50...
🔄 Mengambil data dari offset 100...
🔄 Mengambil data dari offset 150...
🔄 Mengambil data dari offset 200...
🔄 Mengambil data dari offset 250...
🔄 Mengambil data dari offset 300...
🔄 Mengambil data dari offset 350...
🔄 Mengambil data dari offset 400...
🔄 Mengambil data dari offset 450...
🔄 Mengambil data dari offset 500...
🔄 Mengambil data dari offset 550...
🔄 Mengambil data dari offset 600...
🔄 Mengambil data dari offset 650...
🔄 Mengambil data dari offset 700...
🔄 Mengambil data dari offset 750...
🔄 Mengambil data dari offset 800...
🔄 Mengambil data dari offset 850...
🔄 Mengambil data dari offset 900...
🔄 Mengambil data dari offset 950...
🔄 Mengambil data dari offset 1000...
🔄 Mengambil data dari offset 1050...
🔄 Mengambil data dari offset 1100...
🔄 Mengambil data dari offset 1150...
🔄 Mengambil data dari offset 1200...
🔄 Mengambil data dari offset 1250...
🔄 Mengambil data dari offset 1300...
🔄 Mengambil data dari of